In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import urllib
%matplotlib inline

url = "https://raw.githubusercontent.com/Elixeus/PUI2015_xwang/master/plotstyle.json"
response = urllib.urlopen(url)
s = json.loads(response.read())
plt.rcParams.update(s)

import geopy
'''
Here you can import all the geocoders you like.
Use dir(geopy.geocoders) to see all the options.
'''
from geopy.geocoders import Nominatim, ArcGIS, GoogleV3, GeoNames

/Users/eliseowang/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:855: UserWarning: svg.embed_char_paths is deprecated and replaced with svg.fonttype; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [7]:
# read the data you want to geocode. I used my environment variable here. You can use your own.
path = os.getenv('SPATIAL')+'dataforgeocoding.csv'
data = pd.read_csv(path)

In [8]:
data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,CUISINE DESCRIPTION,GRADE,SCORE,MONTH,STREET_ADDRESS,add,str,ADDRESS
0,0,0,0,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462,Bakery,A,2,3,1007 MORRIS PARK AVE,"1007 MORRIS PARK AVE, BRONX, New York 10462",MORRIS PARK AVE,"1007 MORRIS PARK AVE, BRONX, New York 10462"
1,1,1,1,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE ...,11225,Hamburgers,A,8,12,469 FLATBUSH AVENUE ...,"469 FLATBUSH AVENUE, BROOKLYN, New York 11225",FLATBUSH AVENUE ...,"469 FLATBUSH AVENUE, BROOKLYN, New York 11225"
2,2,6,6,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE ...,11225,Hamburgers,B,23,7,469 FLATBUSH AVENUE ...,"469 FLATBUSH AVENUE, BROOKLYN, New York 11225",FLATBUSH AVENUE ...,"469 FLATBUSH AVENUE, BROOKLYN, New York 11225"
3,3,12,12,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET ...,10019,Irish,A,2,9,351 WEST 57 STREET ...,"351 WEST 57 STREET, MANHATTAN, New York 10019",WEST 57TH STREET ...,"351 WEST 57TH STREET, MANHATTAN, New York 10019"
4,4,13,13,40356018,RIVIERA CATERER,BROOKLYN,2780,STILLWELL AVENUE,11224,American,A,5,6,2780 STILLWELL AVENUE,"2780 STILLWELL AVENUE, BROOKLYN, New York 11224",STILLWELL AVENUE,"2780 STILLWELL AVENUE, BROOKLYN, New York 11224"


## Define a function of geocoding

In [11]:
'''
This function takes 5 arguments: data, coder, col, n_init, n_end
data: the pandas dataframe you want to geocode
coder: geocoder of your choice
col: the column containing the address you want to geocode
n_init: the starting index
n_end: the finishing index

'''
def geocoding(fname, coder, col, n_init, n_end):
    data = fname
    # create two rows: 'lat' and 'lon' for the data
    data['lat'] = 0
    data['lon'] = 0
    geolocator = coder
    for i in data[col].index[n_init:n_end+1]:
        try:
            loc = geolocator.geocode(data[col][i])
            data.ix[i, 'lat'] = loc.latitude
            data.ix[i, 'lon'] = loc.longitude
        except:
            print 'number %i Error' %i 
            data.ix[i, 'lat'] = 'Error'
            data.ix[i, 'lon'] = 'Error'
    return data

In [13]:
# geocode
new = geocoding(fname = data, coder = ArcGIS(), col = 'ADDRESS',
                n_init = 0, n_end = 5)

In [17]:
# see the result
new.head(10)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,CUISINE DESCRIPTION,GRADE,SCORE,MONTH,STREET_ADDRESS,add,str,ADDRESS,lat,lon
0,0,0,0,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462,Bakery,A,2,3,1007 MORRIS PARK AVE,"1007 MORRIS PARK AVE, BRONX, New York 10462",MORRIS PARK AVE,"1007 MORRIS PARK AVE, BRONX, New York 10462",40.848267,-73.855953
1,1,1,1,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE ...,11225,Hamburgers,A,8,12,469 FLATBUSH AVENUE ...,"469 FLATBUSH AVENUE, BROOKLYN, New York 11225",FLATBUSH AVENUE ...,"469 FLATBUSH AVENUE, BROOKLYN, New York 11225",40.662873,-73.962181
2,2,6,6,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE ...,11225,Hamburgers,B,23,7,469 FLATBUSH AVENUE ...,"469 FLATBUSH AVENUE, BROOKLYN, New York 11225",FLATBUSH AVENUE ...,"469 FLATBUSH AVENUE, BROOKLYN, New York 11225",40.662873,-73.962181
3,3,12,12,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET ...,10019,Irish,A,2,9,351 WEST 57 STREET ...,"351 WEST 57 STREET, MANHATTAN, New York 10019",WEST 57TH STREET ...,"351 WEST 57TH STREET, MANHATTAN, New York 10019",40.767616,-73.984929
4,4,13,13,40356018,RIVIERA CATERER,BROOKLYN,2780,STILLWELL AVENUE,11224,American,A,5,6,2780 STILLWELL AVENUE,"2780 STILLWELL AVENUE, BROOKLYN, New York 11224",STILLWELL AVENUE,"2780 STILLWELL AVENUE, BROOKLYN, New York 11224",40.579554,-73.982056
5,5,14,14,40356151,BRUNOS ON THE BOULEVARD,QUEENS,8825,ASTORIA BOULEVARD,11369,American,B,16,11,8825 ASTORIA BOULEVARD,"8825 ASTORIA BOULEVARD, QUEENS, New York 11369",ASTORIA BOULEVARD,"8825 ASTORIA BOULEVARD, QUEENS, New York 11369",40.763998,-73.880486
6,6,24,24,40356151,BRUNOS ON THE BOULEVARD,QUEENS,8825,ASTORIA BOULEVARD,11369,American,A,10,5,8825 ASTORIA BOULEVARD,"8825 ASTORIA BOULEVARD, QUEENS, New York 11369",ASTORIA BOULEVARD,"8825 ASTORIA BOULEVARD, QUEENS, New York 11369",0.000000,0.000000
7,7,33,33,40356483,WILKEN'S FINE FOOD,BROOKLYN,7114,AVENUE U,11234,Delicatessen,A,10,5,7114 AVENUE U,"7114 AVENUE U, BROOKLYN, New York 11234",AVENUE U,"7114 AVENUE U, BROOKLYN, New York 11234",0.000000,0.000000
8,8,36,36,40356483,WILKEN'S FINE FOOD,BROOKLYN,7114,AVENUE U,11234,Delicatessen,A,10,1,7114 AVENUE U,"7114 AVENUE U, BROOKLYN, New York 11234",AVENUE U,"7114 AVENUE U, BROOKLYN, New York 11234",0.000000,0.000000
9,9,39,39,40356649,REGINA CATERERS,BROOKLYN,6409,11 AVENUE ...,11219,American,A,12,7,6409 11 AVENUE ...,"6409 11 AVENUE, BROOKLYN, New York 11219",11ST AVENUE ...,"6409 11ST AVENUE, BROOKLYN, New York 11219",0.000000,0.000000
